In [ ]:
edges = {(0, 1): 1, (1, 2): 1, (2, 3): 1, (0, 3): 1}
num_nodes = 4

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Imprime un grafo dirigido pesado
# Entrada
#     graph_nodes: Lista de nodos del grafo
#     graph_edges: Diccionario {vértice: peso} donde vértice es (n1, n2)
#     layout:      Para especificar la posición de los nodos
def print_graph(graph_nodes, graph_edges, layout=None):
    G = nx.DiGraph()

    # Nodes
    G.add_nodes_from(graph_nodes)

    # Edges
    for pair, weight in graph_edges.items():
        G.add_edge(pair[0], pair[1])

    # Print graph
    if layout is None:
        layout = nx.spring_layout(G)

    plt.figure(3, figsize=(6, 2))
    nx.draw(G, layout, with_labels=True, node_size=600)
    edge_labels = nx.get_edge_attributes(G, "weight")
    nx.draw_networkx_edge_labels(G, pos=layout, edge_labels=edge_labels)
    #plt.savefig("../../latex/img/primer_grafo/primer_grafo.png")

# edges = {(0, 1): "5(q_0)", (0, 2): "8(q_1)", (1, 2): "2(q_2)", (1, 3): "7(q_3)", (2, 3): "4(q_4)"}
print_graph(range(num_nodes), edges, layout={0: [-1, 0], 1: [0, 1],
                                             2: [1, 0],  3: [0, -1]})

In [ ]:
# Función de coste binaria con valores {0, 1}
cost_function = "\
-(x_0*(1-x_1) + x_1*(1-x_0) + \
x_1*(1-x_2) + x_2*(1-x_1) + \
x_2*(1-x_3) + x_3*(1-x_2) + \
x_3*(1-x_0) + x_0*(1-x_3))"

# Evalúa una solución concreta según la función de coste para la versión QUBO
# Entrada:
#     - bits: Cadena con los valores de los qubits medidos
#             Orden: q_4 q_3 q_2 q_1 q_0
def eval_cost_function(bits):
    assert num_nodes == len(bits), "Error in cost_function: Length of bits"

    inv_bits = bits[::-1]  # Los qubits están en orden inverso

    param_dict = {}
    # Asignación entre x_ij -> q_n
    # Establecido por el orden de edges. Ej: x_12 -> q_2 (esto es, qubit nº2)
    for i in range(num_nodes):
        param_dict[f"x_{i}"] = int(inv_bits[i])

    return eval(cost_function, param_dict)

In [ ]:
# Hallar mínimo de la función de manera clásica por fuerza bruta
import itertools

for z in itertools.product([0, 1], repeat=num_nodes):
    param_dict = dict()
    for i in range(num_nodes):
        param_dict[f"x_{i}"] = z[i]

    print(z, eval(cost_function, param_dict))

In [ ]:
from qiskit import QuantumCircuit

def generate_qaoa_circuit(theta):
    assert len(theta) % 2 == 0, "Error in parameters (Beta, Gamma)"

    nqubits = num_nodes
    circuit = QuantumCircuit(nqubits)

    layers = int(len(theta) / 2)
    beta = theta[:layers]
    gamma = theta[layers:]

    # |v0>
    for i in range(nqubits):
        circuit.h(i)

    for p in range(layers):
        circuit.barrier()

        # Hp
        for q_idxs, _ in edges.items():
            circuit.rzz(-gamma[p], q_idxs[0], q_idxs[1])

        circuit.barrier()

        # Hm
        for q_idx in range(nqubits):
            circuit.rx(beta[p] * 2, q_idx)

    circuit.measure_all()
    return circuit

In [ ]:
def compute_expectation(counts):
    media = 0
    len_count = 0
    for bits, count in counts.items():
        cost = eval_cost_function(bits)
        media += cost * count
        len_count += count

    return media/len_count

In [ ]:
# Simulador
from qiskit import Aer
from scipy.optimize import minimize

backend = Aer.get_backend('aer_simulator')
shots = 1024

def execute_circuit(theta):
    qc = generate_qaoa_circuit(theta)
    counts = backend.run(qc, shots=shots).result().get_counts()
    return compute_expectation(counts)

num_layers = 1
theta_res = minimize(execute_circuit, [1.0, 1.0] * num_layers, method = "COBYLA")
theta_res

In [ ]:
from qiskit.visualization import plot_histogram
backend = Aer.get_backend('aer_simulator')
shots = 1024

qc = generate_qaoa_circuit(theta_res.x)
counts = backend.run(qc, shots=shots).result().get_counts()

normalized_counts = {key: val / shots for (key, val) in counts.items()}
plot_histogram(normalized_counts, figsize=(13, 6.83))

In [ ]:
# Gamma function
import numpy as np
import matplotlib.pyplot as plt

def gamma_function():
    x = np.linspace(0.3, 6, 300)
    y = []
    beta = 1.0
    for gamma in x:
        y.append(execute_circuit([beta, gamma]))

    plt.plot(x, y)
    # plt.savefig("../../latex/resultados/img/primer_grafo/sin_restriccion_extra/primer_paper_p_27_gamma_fun.png")
    plt.show()

gamma_function()

In [ ]:
# 3D Gamma function
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import itertools

def gamma_function_3d():
    beta = np.arange(0.3, 6, (6-0.3)/120)
    gamma = np.arange(0.3, 6, (6-0.3)/120)
    beta, gamma = np.meshgrid(beta, gamma)
    z = np.zeros([len(beta), len(beta)])

    for i in range(len(beta)):
        for j in range(len(beta)):
            z[i][j] = execute_circuit([beta[i][j], gamma[i][j]])

    fig = plt.figure()
    axis = plt.axes(projection='3d')
    figure = axis.plot_surface(gamma, beta, z, cmap=cm.coolwarm, linewidth=0, antialiased=False)
    fig.colorbar(figure, shrink=0.5, aspect=5)
    # plt.savefig("../../latex/resultados/img/primer_grafo/sin_restriccion_extra/primer_paper_p_27_gamma_fun.png")
    plt.show()

gamma_function_3d()

In [ ]:
# Max statistics
def max_statistics(num_layers=1, num_generations=1000):
    statistics = {}
    for generation in range(0, num_generations):
        theta_res = minimize(execute_circuit, [1.0, 1.0] * num_layers, method = "COBYLA")
        qc = generate_qaoa_circuit(theta_res.x)
        counts = backend.run(qc, shots=shots).result().get_counts()

        path = max(counts, key=counts.get)
        if path not in statistics:
            statistics[path] = 0
        statistics[path] += 1

    for path in statistics:
        statistics[path] = statistics[path] / num_generations  # Normalize
    return statistics

max_statistics(num_layers=3, num_generations=1000)

In [ ]:
# Global statistics
def global_statistics(num_layers=1, num_generations=1000):
    statistics = {}
    for iteration in range(0, num_generations):
        theta_res = minimize(execute_circuit, [1.0, 1.0] * num_layers, method = "COBYLA")
        qc = generate_qaoa_circuit(theta_res.x)
        counts = backend.run(qc, shots=shots).result().get_counts()

        sorted_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
        for (path, num_appearances) in sorted_counts:
            if path not in statistics:
                statistics[path] = 0
            statistics[path] += num_appearances

    for path in statistics:
        statistics[path] = statistics[path] / shots / num_generations  # Normalize

    statistics = sorted(statistics.items(), key=lambda x: x[1], reverse=True)
    return statistics

global_statistics(num_layers=3, num_generations=1000)

In [ ]:
# p = 1
# MAX: ('0101': 0.51), ('1010': 0.49)
# GLOBAL: ('1010', 0.260873046875) ('0101', 0.2605244140625) ('1001', 0.087294921875) ...
# p = 2
# MAX: ('1010': 0.507), ('0101': 0.493)
# GLOBAL: ('1010', 0.49141796875) ('0101', 0.4902607421875) ('0011', 0.003029296875) ...
# p = 3
# MAX: ('0101': 0.47), ('1010': 0.53)
# GLOBAL: ('0101', 0.481150390625) ('1010', 0.4798994140625) ('0110', 0.0069404296875) ...

In [ ]:
!dunstify -t $((30* 1000*60)) --urgency=critical "Ejecución completada"